<a href="https://colab.research.google.com/github/pleunipennings/CSC508_ML_Biomedicine_Class/blob/main/Module4/ExtraCredit_Mod4_2_ReadingEHRData_creatingPatData.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Alzheimer's data, creating PatData.csv

Many people are affected by Alzheimers (the Alzheimer’s Association estimates 6 million patients and 11 million unpaid caregivers in the US, see [link](https://www.alz.org/alzheimers-dementia/facts-figures))

Here is some information, written by Dr Raven Baxter [link text](https://threadreaderapp.com/thread/1380339786515423232.html)

Have a look at this short video on [youtube](https://www.youtube.com/watch?v=Eq_Er-tqPsA)

# Having a first look at the data 

The dataset we are using comes from the ADNI project, a huge project to gather data on people with Alzheimer’s disease and healthy controls. 

## Opening the file location and loading libraries

Import libraries

In [ ]:
import pandas as pd
from pandas.api.types import CategoricalDtype
import numpy as np
from google.colab import files #For exporting files from google colab notebook

Read the dataset "TADPOLE_D1_D2_57columns.csv" from GitHib

The full repo is here https://github.com/pleunipennings/CSC508Data


In [ ]:
url = "https://raw.githubusercontent.com/pleunipennings/CSC508Data/main/TADPOLE_D1_D2_57columns.csv" 
data = pd.read_csv(url)

Look at the first few rows of the data

In [ ]:
data.head()

Find out how big is the data?

In [ ]:
data.shape

Two columns that are important are **PTID** and **EXAMDATE**. 
PTID is the patient ID and EXAMDATE is –  you guessed it – the date at which the medical exam took place! 

Let's take a look at one patient with the ID: 011_S_0003

In [ ]:
data.loc[data['PTID'] == '011_S_0003']

## Question 1: Looking at the data

In which years did this patient come in for exams?  
What is the age, marital status and race of the patient?


# Categoricals and Ordered Categories! 


Some examples of categorical data are gender, social class, blood type, or rating scale.

Here, we're going to make the column Diagnosis 'DX' into a categorical data type and specify an order on the categories. We need to do this so that we can use the ordering to select the data we need. 

Here are what the abbreviations are:
*   **NL** = no Alzheimers (cognitive normal)
*   **MCI** = Mild Cognitive Impairement
*   **Dementia** = has Alzheimers

Here is a guide for Categorical data types: https://pandas.pydata.org/pandas-docs/stable/user_guide/categorical.html#categoricaldtype


In [ ]:
DX_type = CategoricalDtype(categories=['NL', 'NL to MCI', 'MCI to NL', 'MCI', 
                           'Dementia to MCI', 'MCI to Dementia', 
                           'Dementia', 'NL to Dementia'], ordered=True)
data['DX'] = data['DX'].astype(DX_type)

## Question 2: Ordered Categories

Create ordered categories for the column 'DX_bl' (diagnosis at baseline). The levels are:
*   **CN**: cognitive normal
*   **SMC**: subjective memory concerns
*   **EMCI**: Early Mild Cognitive Impairment
*   **LMCI**: Late Mild Cognitive Impairment
*   **AD**: Alzheimer’s Disease

Check that your code worked. 
Take a screenshot of your code and include it in the pdf or wordfile you submit on iLearn. 


In [ ]:
# Put your code here and take a screenshot to include in your word file to hand in on iLearn. 


# Grouping data: One row per patient

For some types of analysis, it is useful to have just one row per patient instead of multiple. 

Let's create a dataset that has just one row per patient. To do so, we need to 'group' the data based on something unique. In this dataset, we can use the Patient ID and group all the entries that have the same ID. Then we can aggregate (this means to combine the data) and grab the max entry or calculate the mean for the groupings that are the same. 

Note: Real-life data is always messy and may be missing. So we want to make sure we use the pandas dropna argument and set it to False, otherwise all entries with 'NA's will be removed. You'll actually be missing important patient data! 

Guide for Summarising, Aggregating, Grouping data in Python: https://www.shanelynn.ie/summarising-aggregation-and-grouping-data-in-python-pandas/

Guide for handling missing data: https://machinelearningmastery.com/handle-missing-data-python/

In [ ]:
Pat_Data = data.groupby(
    ['PTID', 'PTGENDER', 'PTEDUCAT','PTETHCAT', 'PTRACCAT','PTMARRY','APOE4'], dropna=False
    ).aggregate(
        {'DX': 'max',
         'AGE': 'median',
         'Ventricles': 'mean',
         'Hippocampus': 'mean',
         'WholeBrain': 'mean', 
         'Entorhinal': 'mean', 
         'Fusiform': 'mean', 
         'MidTemp': 'mean', 
         'ICV': 'mean'}
          ).reset_index()

In [ ]:
Pat_Data.shape

In [ ]:
#Let's see what happened to the patient we looked at earlier. Patient 011_S_0003. 
Pat_Data.loc[Pat_Data['PTID'] == '011_S_0003']

#Assigment 3: shape of the dataframe
How did the shape of the dataframe change? How many rows did the original datafram have per patient approximately? 

#More categorical variables

What if we had multiple columns that were categorical? Here, we can make multiple columns into categoricals by using the lambda function. A lambda function can take any number of arguments, but can only have one expression.


Here are some guides on the lambda function and the apply function
*   lambda function: https://www.w3schools.com/python/python_lambda.asp

*   apply function: https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.apply.html?highlight=apply






Now, we can take a look at the data type of each column. We can check to see if the columns we have changed (DX, DX_bl, PTGENDER, PTETHCAT, PTRACCAT, and PTMARRY) are correct and listed as 'category'. 

In [ ]:
cols = ['PTGENDER', 'PTETHCAT', 'PTRACCAT', 'PTMARRY']
data[cols] = data[cols].apply(lambda x: x.astype('category'))

In [ ]:
data.dtypes

# Summary Statistics

You may be curious about the patients in the study. By using the describe() function, you could take a look at the summary statistics. In this case, we see that there are more males in the study.

*   count = the number of elements in your dataset
*   unique = the total number of unique elements
*   top = most frequent entry
*   freq = Frequency of the most frequent entry

Depending on the column data type, the summary statistics could also include the mean, std, min max, etc

Guide on summary statistics: https://www.askpython.com/python/examples/calculate-summary-statistics

What happens if you want to know the count of all of the cateogories? You can use the value_counts() function. Take a look at the two code chunks below.

In [ ]:
Pat_Data['PTGENDER'].describe()

In [ ]:
Pat_Data['PTGENDER'].value_counts()

## Assignment 4: Summary

After you created PatData, use head() to have a look at the first rows.  

1.   Look at the summary statistics for race ('PTRACCAT'). What is the most frequent answer? Use describe() and value_counts(). Which function do you like best? 
2.   Try Pat_Data.describe() and look at the summary statistics for the whole dataset. What is the average age? What is the age of the youngest patient? What is the age of the oldest patient? Does it show all variables? 





# Exporting as a csv file

Save the newly created dataset that contains one row per patient.
This will be saved onto your Desktop.
Replace "YourVersion" by your own name so that you know that this file was made by you. 

In [ ]:
Pat_Data.to_csv('PatData_YourVersion.csv')
files.download('PatData_YourVersion.csv')